In [1]:
# add parent directory to path: enable import from parent dir
import sys
sys.path.append('../')

from environment import SmartBrokerEnv
from agents.sac import SAC
from networks.sac.continuous.policy_net_l import PolicyNetworkLSTM

import matplotlib.pyplot as plt

In [2]:
norm_cols = ['Volume XRP']
cols = ['date'] + ['open', 'high', 'low', 'close', f'Volume XRP']
batch_dur = 20
env = SmartBrokerEnv(
    batch_dur=batch_dur,
    df_info={
        'start_date': '2021-01-01',
        'end_date': '2021-02-01',
        'norm_cols': norm_cols,
        'cols': cols,
    },
    portfolio={

    },
)
sac = SAC(
    env=env,
    name='smart_broker',
    input_dim=env.observation_space.shape[0],
    log_freq=5,
    hyprprms={
        'mem_sz': 500,
        'lr': 0.0004,
        'batch_size': 1,

    },
    networks={
        'policy_net': PolicyNetworkLSTM(
            state_dim=env.observation_space.shape[0],
            action_dim=env.action_space.shape[0],
            eps=1e-6,
        ),
    },
    p_net_type='lstm',
)

/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/core/frame.py:3641: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/Users/vinay/code/git/smart-broker/notebooks/../environment.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df['rolling_price'] = self.df[self.price_typ].rolling(self.roll_period).sum()
/Users/vinay/miniforge3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataF

In [3]:
sac.run(5000)

collecting experience...
0..
Ep: 5 | TS: 2820 | L: -2742.35 | R: 0.55 | R.Avg.R: 0.54 | P: 2.19 | R.Avg P: 1.66 | B: 53.2 | R.Avg B: 52.87 | R.Avg.U: 210
Ep: 10 | TS: 5170 | L: -4503.59 | R: 0.56 | R.Avg.R: 0.54 | P: 4.45 | R.Avg P: 2.25 | B: 53.84 | R.Avg B: 52.04 | R.Avg.U: 216
Ep: 15 | TS: 7520 | L: -5519.77 | R: 0.54 | R.Avg.R: 0.54 | P: 1.81 | R.Avg P: 2.18 | B: 50.64 | R.Avg B: 52.62 | R.Avg.U: 214
Ep: 20 | TS: 9870 | L: -6143.74 | R: 0.54 | R.Avg.R: 0.54 | P: 1.63 | R.Avg P: 2.19 | B: 54.67 | R.Avg B: 53.22 | R.Avg.U: 211

KeyboardInterrupt: 

## Visualizations

### Training

In [ ]:
fig, ax = plt.subplots(3, 2, figsize=(16, 12))
r_avg_rewards = []
r_avg_profits = []
r_avg_bal = []
r_avg_units_held = []
r_avg_loss = []
r_avg_net_worth = []
count = 0

for _, log in sac.logs.items():
    r_avg_rewards.append(log['r_avg_reward'])
    r_avg_profits.append(log['r_avg_profit'])
    r_avg_bal.append(log['r_avg_bal'])
    r_avg_units_held.append(log['r_avg_units_held'])
    r_avg_loss.append(log['r_avg_loss'])
    r_avg_net_worth.append(log['r_avg_net_worth'])
    count += 1

ax[0][0].plot(range(count), r_avg_loss)
ax[0][0].set_title('Rolling avg loss per episode')

ax[0][1].plot(range(count), r_avg_rewards)
ax[0][1].set_title('Rolling avg reward per episode')

ax[1][0].plot(range(count), r_avg_profits)
ax[1][0].set_title('Rolling avg profit per episode')

ax[1][1].plot(range(count), r_avg_units_held)
ax[1][1].set_title('Rolling avg units held per episode')

ax[2][0].plot(range(count), r_avg_net_worth)
ax[2][0].set_title('Rolling avg net worth per episode')

ax[2][1].plot(range(count), r_avg_bal)
ax[2][1].set_title('Rolling avg balance per episode')